# McMaster Optimal Aging Portal

McMaster Optimal Aging Portal blog

### Blog posts
Scraping of blog posts at https://www.mcmasteroptimalaging.org/blog/1

In [2]:
import requests
import pandas as pd
import io
from bs4 import BeautifulSoup

In [7]:
# URLs for blog posts
url_main = "https://www.mcmasteroptimalaging.org/blog/"
pages = [i for i in range(1, 20)] # Pages 1 to 19
url_list = [url_main + str(page) for page in pages]
url_list

['https://www.mcmasteroptimalaging.org/blog/1',
 'https://www.mcmasteroptimalaging.org/blog/2',
 'https://www.mcmasteroptimalaging.org/blog/3',
 'https://www.mcmasteroptimalaging.org/blog/4',
 'https://www.mcmasteroptimalaging.org/blog/5',
 'https://www.mcmasteroptimalaging.org/blog/6',
 'https://www.mcmasteroptimalaging.org/blog/7',
 'https://www.mcmasteroptimalaging.org/blog/8',
 'https://www.mcmasteroptimalaging.org/blog/9',
 'https://www.mcmasteroptimalaging.org/blog/10',
 'https://www.mcmasteroptimalaging.org/blog/11',
 'https://www.mcmasteroptimalaging.org/blog/12',
 'https://www.mcmasteroptimalaging.org/blog/13',
 'https://www.mcmasteroptimalaging.org/blog/14',
 'https://www.mcmasteroptimalaging.org/blog/15',
 'https://www.mcmasteroptimalaging.org/blog/16',
 'https://www.mcmasteroptimalaging.org/blog/17',
 'https://www.mcmasteroptimalaging.org/blog/18',
 'https://www.mcmasteroptimalaging.org/blog/19']

In [8]:
posts = {}
posts['links'] = []
posts['dates'] = []
posts['header'] = []

for url in url_list:
  blog_page = requests.get(url)
  soup = BeautifulSoup(blog_page.content, 'html.parser')
  blog_headers = soup.select('h2.sfpostTitle a')
  posts['links'].append([link['href'] for link in list(blog_headers)])
  posts['header'].append([header.get_text(strip = True) for header in list(blog_headers)])

  blog_date = soup.find_all('div', class_ = 'monthyear')
  posts['dates'].append([date.get_text(strip = True) for date in list(blog_date)])

In [9]:
# Unnesting news links, dates, and headers
links = []
dates = []
headers = []

for urls in posts['links']:
  for link in urls:
    links.append(link)

for dates_ in posts['dates']:
  for date in dates_:
    dates.append(date)

for headers_ in posts['header']:
  for header in headers_:
    headers.append(header)

In [10]:
posts['links'] = links
posts['dates'] = dates
posts['header'] = headers
pd.DataFrame(posts)

,links,dates,header
0,https://www.mcmasteroptimalaging.org/blog/deta...,16 February 2022,Collaboration is key: Patient engagement for h...
1,https://www.mcmasteroptimalaging.org/blog/deta...,16 February 2022,Dandruff or seborrheic dermatitis? An itch by ...
2,https://www.mcmasteroptimalaging.org/blog/deta...,9 February 2022,The other side of the screen
3,https://www.mcmasteroptimalaging.org/blog/deta...,2 February 2022,Upping fruit and veggie consumption to get blo...
4,https://www.mcmasteroptimalaging.org/blog/deta...,2 February 2022,Vitamin D: Should I be taking a supplement?
...,...,...,...
374,https://www.mcmasteroptimalaging.org/blog/deta...,15 March 2014,Loneliness hurts. How to recognize loneliness ...
375,https://www.mcmasteroptimalaging.org/blog/deta...,14 March 2014,Recognize and prevent delirium (confusion and ...
376,https://www.mcmasteroptimalaging.org/blog/deta...,9 March 2014,Social isolation: what is it and why is it imp...
377,https://www.mcmasteroptimalaging.org/blog/deta...,8 March 2014,Multimorbidity: how to improve quality of life...


In [11]:
test_urls = posts['links'][0:3]
test_urls

['https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/02/17/collaboration-is-key-patient-engagement-for-healthcare-improvement',
 'https://www.mcmasteroptimalaging.org/blog/detail/videos/2022/02/16/dandruff-or-seborrheic-dermatitis-an-itch-by-any-other-name',
 'https://www.mcmasteroptimalaging.org/blog/detail/blog/2022/02/09/the-other-side-of-the-screen']

In [15]:
posts['body'] = []
for url in posts['links']:
  blog_post = requests.get(url)
  soup = BeautifulSoup(blog_post.content, "html.parser")

  # find_all method to gather article body
  blog_body = soup.find_all('div', itemprop = "articleBody")
  
  # find_all method to gather text content
  body = []
  for paragraph in blog_body:
    blog_p = paragraph.find_all('p')
    body.append(' '.join([text.get_text(strip = True) for text in list(blog_p) if text.get_text(strip = True) != '']))

  # Populating posts dictionary
  posts['body'].append(body)

In [18]:
len(posts['body'])

379

In [19]:
pd.DataFrame(posts)

,links,dates,header,body
0,https://www.mcmasteroptimalaging.org/blog/deta...,16 February 2022,Collaboration is key: Patient engagement for h...,[Have you ever heard the sayings “Teamwork mak...
1,https://www.mcmasteroptimalaging.org/blog/deta...,16 February 2022,Dandruff or seborrheic dermatitis? An itch by ...,[Seborrheic dermatitis is a common rash charac...
2,https://www.mcmasteroptimalaging.org/blog/deta...,9 February 2022,The other side of the screen,[The COVID-19 pandemic and the pandemic respon...
3,https://www.mcmasteroptimalaging.org/blog/deta...,2 February 2022,Upping fruit and veggie consumption to get blo...,[Blood pressure is a term we may hear often bu...
4,https://www.mcmasteroptimalaging.org/blog/deta...,2 February 2022,Vitamin D: Should I be taking a supplement?,[Vitamin D plays an important role in many fun...
...,...,...,...,...
374,https://www.mcmasteroptimalaging.org/blog/deta...,15 March 2014,Loneliness hurts. How to recognize loneliness ...,"[Most people feel lonely once in a while, but ..."
375,https://www.mcmasteroptimalaging.org/blog/deta...,14 March 2014,Recognize and prevent delirium (confusion and ...,"[Delirium, sometimes called acute confusional ..."
376,https://www.mcmasteroptimalaging.org/blog/deta...,9 March 2014,Social isolation: what is it and why is it imp...,[Human beings are social animals. We live with...
377,https://www.mcmasteroptimalaging.org/blog/deta...,8 March 2014,Multimorbidity: how to improve quality of life...,"[At the most recent visit to my doctor, he men..."


In [20]:
pd.DataFrame(posts).to_csv('mcmaster_blog_posts.csv', encoding = 'utf-8-sig')